In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import csv


class MyDataset(object):
    '''
    Class to load the dataset
    img_dir = Plots dir
    ground_dir = Ground dir where:
        column 1 = Plot name
        column 2 = Plot ground value
        column 3 = Plot GDD value
    '''
    def __init__(self, img_dir, ground_dir):
        self.img_dir = img_dir
        self.ground_dir = ground_dir
        self.df = self._get_img_df()
        self.ground_arr = self._get_ground_df()
        self.GDD_arr = self._get_GDD_df()
    
    def __len__(self):
        return len(self.df)

    def _get_img_df(self):
        img_names = os.listdir(self.img_dir)
        df = pd.DataFrame(img_names, columns=['Filename'])
        df['Global_ID'] = df.Filename.str[6:-8].astype('int') ###[Global_ID] & [6:-8] to be adjusted
        df['DOY'] = df.Filename.str[-7:-4].astype('int') ###[-7:-4] to be adjusted
        df = pd.pivot_table(df, values = 'Filename', index='Global_ID', columns='DOY', aggfunc='first').reset_index() ###[Global_ID] to be adjusted
        df= df.sort_values('Global_ID')
        return(df)

    def _get_ground_df(self):
        ground_arr =  pd.read_csv(self.ground_dir, usecols=(0,1))
        ground_arr = pd.DataFrame(ground_arr)
        ground_arr = ground_arr.sort_values('Global_ID')
        ground_arr = ground_arr.to_numpy()
        return(ground_arr)

    def _get_GDD_df(self):
        GDD_arr = pd.read_csv(self.ground_dir, usecols=(0,3))
        GDD_arr = pd.DataFrame(GDD_arr)
        GDD_arr = GDD_arr.sort_values('Global_ID')
        GDD_arr = GDD_arr.to_numpy()
        return(GDD_arr)

    def _transform(self, x):
        x = np.moveaxis(x, -3, -2) ###[np.moveaxis(x, -2, -2)] to be adjusted. Dim goals: Plot:Flight:R:G:B
        ### Normalization or rescale did NOT change the results.

        # x = x.astype('float32')

        # layer = Normalization()
        # layer.adapt(x)
        # x = layer(x)
        # x = Rescaling(1.0 / 255)(x)
        # input_shape = x.shape[:2]
        # inputs = keras.Input(input_shape[0],input_shape[1])
        # x = Rescaling(1.0 / 255)(inputs)  # Rescale inputs
        # x /= 255.0

        # min = x.min()
        # max = x.max()
        # x -= min
        # x /= (max - min)
        
        return(x)

    def __getitem__(self,imgs_stacked):
        alist = [] 
        # GDDlist = [20.8595, 20.7809, 20.9571, 20.6049, 21.0528, 20.8987, 21.1707, 21.6270, 21.2946]
        # GDDdate = 0

        for column in tqdm(self.df.columns[1:]): 
            print(f"Column name:",column)
            imgs_stack = np.stack([cv2.imread(os.path.join(self.img_dir, img)) for img in self.df[column]])
            # imgs_stack = np.stack([cv2.cvtColor(cv2.imread(os.path.join(self.img_dir, img)), cv2.COLOR_BGR2HSV) for img in self.df[column]]) ### Want to change the color space? [cv2.COLOR_BGR2HSV]

            imgs_stack = self._transform(imgs_stack)
            # imgs_stack =  imgs_stack*GDDlist[GDDdate]

            alist.append([imgs_stack])
            # GDDdate += 1
        imgs_stacked = np.squeeze(np.stack(alist, axis=2))                                                                                         

        return (imgs_stacked) 
        #display(arr)

In [3]:
#%cd /content/drive/MyDrive/Soybean_maturity_Trevisan/
main_dir = '/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2021/SVREC_Mat'
img_dir = os.path.join(main_dir,'e._ClipPlots_BN','SVREC_P1_512_128')

ground_dir = os.path.join(main_dir,'d._Ground_notes', '2021_SVREC_BN_ground_P01.csv')


In [ ]:
dataset_arr =  MyDataset(img_dir, ground_dir)[object]
len(dataset_arr)

  0%|          | 0/9 [00:00<?, ?it/s]

Column name: 42
Column name: 47
Column name: 56
Column name: 61
Column name: 72
Column name: 77
Column name: 82
Column name: 85
Column name: 89


524

In [ ]:
dataset_arr.shape

(524, 9, 512, 128, 3)

In [ ]:
image = dataset_arr[111,5]
image.shape
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (image.min(), image.max()))

Min: 0.000, Max: 255.000


In [ ]:
image

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)

In [4]:
dataset_df =  MyDataset(img_dir, ground_dir).df
dataset_df
#dataset_df.to_csv('C:\\temp_maturity\\UMNSoybeans_Planting_Season_2018\\df.csv')

DOY,Global_ID,42,47,56,61,72,77,82,85,89
0,21021001,SVREC_21021001_042.png,SVREC_21021001_047.png,SVREC_21021001_056.png,SVREC_21021001_061.png,SVREC_21021001_072.png,SVREC_21021001_077.png,SVREC_21021001_082.png,SVREC_21021001_085.png,SVREC_21021001_089.png
1,21021002,SVREC_21021002_042.png,SVREC_21021002_047.png,SVREC_21021002_056.png,SVREC_21021002_061.png,SVREC_21021002_072.png,SVREC_21021002_077.png,SVREC_21021002_082.png,SVREC_21021002_085.png,SVREC_21021002_089.png
2,21021003,SVREC_21021003_042.png,SVREC_21021003_047.png,SVREC_21021003_056.png,SVREC_21021003_061.png,SVREC_21021003_072.png,SVREC_21021003_077.png,SVREC_21021003_082.png,SVREC_21021003_085.png,SVREC_21021003_089.png
3,21021004,SVREC_21021004_042.png,SVREC_21021004_047.png,SVREC_21021004_056.png,SVREC_21021004_061.png,SVREC_21021004_072.png,SVREC_21021004_077.png,SVREC_21021004_082.png,SVREC_21021004_085.png,SVREC_21021004_089.png
4,21021005,SVREC_21021005_042.png,SVREC_21021005_047.png,SVREC_21021005_056.png,SVREC_21021005_061.png,SVREC_21021005_072.png,SVREC_21021005_077.png,SVREC_21021005_082.png,SVREC_21021005_085.png,SVREC_21021005_089.png
...,...,...,...,...,...,...,...,...,...,...
519,21053036,SVREC_21053036_042.png,SVREC_21053036_047.png,SVREC_21053036_056.png,SVREC_21053036_061.png,SVREC_21053036_072.png,SVREC_21053036_077.png,SVREC_21053036_082.png,SVREC_21053036_085.png,SVREC_21053036_089.png
520,21053037,SVREC_21053037_042.png,SVREC_21053037_047.png,SVREC_21053037_056.png,SVREC_21053037_061.png,SVREC_21053037_072.png,SVREC_21053037_077.png,SVREC_21053037_082.png,SVREC_21053037_085.png,SVREC_21053037_089.png
521,21053038,SVREC_21053038_042.png,SVREC_21053038_047.png,SVREC_21053038_056.png,SVREC_21053038_061.png,SVREC_21053038_072.png,SVREC_21053038_077.png,SVREC_21053038_082.png,SVREC_21053038_085.png,SVREC_21053038_089.png
522,21053039,SVREC_21053039_042.png,SVREC_21053039_047.png,SVREC_21053039_056.png,SVREC_21053039_061.png,SVREC_21053039_072.png,SVREC_21053039_077.png,SVREC_21053039_082.png,SVREC_21053039_085.png,SVREC_21053039_089.png


In [5]:
dataset_ground =  MyDataset(img_dir, ground_dir).ground_arr
dataset_ground

array([[21021001,       24],
       [21021002,       24],
       [21021003,       26],
       ...,
       [21053038,       32],
       [21053039,       29],
       [21053040,       26]])

In [6]:
dataset_GDD =  MyDataset(img_dir, ground_dir).GDD_arr
dataset_GDD

array([[2.10210010e+07, 2.11707317e+01],
       [2.10210020e+07, 2.11707317e+01],
       [2.10210030e+07, 2.13166667e+01],
       ...,
       [2.10530380e+07, 2.15700000e+01],
       [2.10530390e+07, 2.15494253e+01],
       [2.10530400e+07, 2.13166667e+01]])

In [ ]:
# Change the current working directory
#os.chdir('H:\\My Drive\\UMNSoybeans_Planting_Season_2018\\e._npy_data')
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2021\\SVREC_Mat\\f._numpy_data\\SVREC_P1')
np.save('2021_SVREC_9fly_BN_512_128.npy', dataset_arr)  

In [ ]:
# Change the current working directory
os.chdir('C:\\Users\\leoag\\Michigan State University\\MSU Dry Bean Breeding Lab - General\\UAS_Beans\\2021\\SVREC_Mat\\f._numpy_data\\SVREC_P1')
np.save('2021_SVREC_ground_9fly_BN_1024_264.npy', dataset_ground)  

In [7]:
# Change the current working directory
os.chdir('/content/drive/MyDrive/UAS_Beans/Beans_Maturity/2021/SVREC_Mat/f._numpy_data/SVREC_P1')
np.save('2021_SVREC_GDD_BN.npy', dataset_GDD) 